In [ ]:
# -*- coding: utf-8 -*-
"""skin_tone.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1z6Lq_jkMTalP4mHFs1Sq_vb2RadFSb8Q
"""

from zipfile import ZipFile
import os

# Upload archive.zip manually via Colab file browser first
zip_path = "archive.zip"
extract_path = "/content/skintone_dataset"

with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check structure
os.listdir(extract_path)

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

train_path = '/content/skintone_dataset/train'

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * np.exp(-0.1)

def build_skin_tone_model():
    model = Sequential([
        Input(shape=(224, 224, 3)),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(512, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')  # 3 classes: Black, Brown, White
    ])
    return model

model = build_skin_tone_model()

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    LearningRateScheduler(lr_scheduler),
    EarlyStopping(patience=5, restore_best_weights=True)
]

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

# Evaluate the Model
val_loss, val_acc = model.evaluate(val_generator, verbose=1)
print(f"Validation Accuracy: {val_acc:.2%}")

# Define the path inside your Google Drive
save_path = "vgg16_skin_tone.h5"

# Save the model
model.save(save_path)

print(f"Model saved to: {save_path}")

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# ========== 1. Extract Dataset ==========
from zipfile import ZipFile
import os

zip_path = "archive.zip"  # Upload via Colab sidebar
extract_path = "/content/skintone_dataset"

with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extracted folders/files:", os.listdir(extract_path))


# ========== 2. Import Libraries ==========
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import VGG16
from sklearn.utils.class_weight import compute_class_weight


# ========== 3. Data Augmentation ==========
train_path = os.path.join(extract_path, 'train')

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.7, 1.4],
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


# ========== 4. Handle Class Imbalance ==========
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))


# ========== 5. Load Pretrained VGG16 Model ==========
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
base_model.trainable = False  # Freeze pretrained layers

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # 3 skin tone classes

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# ========== 6. Callbacks ==========
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1)
]


# ========== 7. Training ==========
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)


# ========== 8. Evaluation ==========
val_loss, val_acc = model.evaluate(val_generator, verbose=1)
print(f"✅ Final Validation Accuracy: {val_acc:.2%}")


# ========== 9. Visualization ==========
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()



from zipfile import ZipFile
import os

zip_path = "archive.zip"  # replace with new uploaded zip filename
extract_path = "/content/skintone_dataset"

with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Extracted folders:", os.listdir(extract_path))

import os
print("Subfolders in /train:", os.listdir(os.path.join(extract_path, "train")))

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from collections import Counter

# ===========================================================
# 3. Data Generators
# ===========================================================
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
VAL_SPLIT = 0.2

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT,
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.7, 1.4],
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    f"{extract_path}/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
)

val_gen = datagen.flow_from_directory(
    f"{extract_path}/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
)

print("\n✅ Class indices:", train_gen.class_indices)
print("📊 Train distribution:", dict(Counter(train_gen.classes)))
print("📊 Val distribution:", dict(Counter(val_gen.classes)))

# ===========================================================
# 4. Compute Class Weights
# ===========================================================
class_weights = dict(enumerate(compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)))

# ===========================================================
# 5. Build VGG16 Model
# ===========================================================
def build_vgg_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=input_shape))
    base_model.trainable = False

    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_vgg_model()
model.summary()

# ===========================================================
# 6. Callbacks
# ===========================================================
callbacks = [
    ModelCheckpoint("best.h5", monitor="val_accuracy", save_best_only=True, mode="max", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, verbose=1),
    EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True, verbose=1)
]

# ===========================================================
# 7. Train the Model
# ===========================================================
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# ===========================================================
# 8. Evaluation and Visualization
# ===========================================================
val_loss, val_acc = model.evaluate(val_gen, verbose=0)
print(f"\n✅ Final validation accuracy: {val_acc:.2%}")

# Plot training curves
def plot_history(hist):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(hist.history['accuracy'], label='Train')
    plt.plot(hist.history['val_accuracy'], label='Validation')
    plt.title("Accuracy")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(hist.history['loss'], label='Train')
    plt.plot(hist.history['val_loss'], label='Validation')
    plt.title("Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_history(history)

# Confusion matrix
val_gen.reset()
preds = model.predict(val_gen, verbose=0)
y_true = val_gen.classes
y_pred = np.argmax(preds, axis=1)

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=["dark", "medium", "light"])
disp.plot(cmap="Blues", colorbar=False)
plt.title("Confusion Matrix – Validation")
plt.show()

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from zipfile import ZipFile
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
VAL_SPLIT = 0.2

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT,
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.7, 1.4],
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    f"{extract_path}/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    f"{extract_path}/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

print("✅ Class indices:", train_gen.class_indices)
print("📊 Train distribution:", dict(Counter(train_gen.classes)))
print("📊 Val distribution:", dict(Counter(val_gen.classes)))

# ===========================================================
# 4. Class Weights
# ===========================================================
class_weights = dict(enumerate(compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)))

# ===========================================================
# 5. Build VGG16 Model (returns model + base_model)
# ===========================================================
def build_vgg16_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=input_shape))
    base_model.trainable = False
    x = Flatten()(base_model.output)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation="softmax")(x)
    model = Model(inputs=base_model.input, outputs=out)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="categorical_crossentropy", metrics=["accuracy"])
    return model, base_model

model, base_model = build_vgg16_model()
model.summary()

# ===========================================================
# 6. Callbacks
# ===========================================================
callbacks = [
    ModelCheckpoint("best.h5", monitor="val_accuracy", save_best_only=True, mode="max", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, verbose=1),
    EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True, verbose=1),
]

# ===========================================================
# 7. Phase 1: Train Frozen VGG16
# ===========================================================
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# ===========================================================
# 8. Phase 2: Fine-Tune Top Layers of VGG16
# ===========================================================
# Unfreeze top 2 convolutional blocks (~last 8 layers)
for layer in base_model.layers[-8:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    initial_epoch=history1.epoch[-1] + 1,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# ===========================================================
# 9. Evaluation and Plots
# ===========================================================
val_loss, val_acc = model.evaluate(val_gen, verbose=0)
print(f"\n✅ Final validation accuracy: {val_acc:.2%}")

def plot_history(histories):
    plt.figure(figsize=(12, 4))
    for h in histories:
        plt.subplot(1, 2, 1)
        plt.plot(h.history['accuracy'])
        plt.plot(h.history['val_accuracy'])
    plt.title("Accuracy")
    plt.legend(['Train', 'Val'])
    plt.grid()

    for h in histories:
        plt.subplot(1, 2, 2)
        plt.plot(h.history['loss'])
        plt.plot(h.history['val_loss'])
    plt.title("Loss")
    plt.legend(['Train', 'Val'])
    plt.grid()
    plt.tight_layout()
    plt.show()

plot_history([history1, history2])

# Confusion matrix
val_gen.reset()
preds = model.predict(val_gen, verbose=0)
y_true = val_gen.classes
y_pred = np.argmax(preds, axis=1)

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=["dark", "medium", "light"])
disp.plot(cmap="Blues", colorbar=False)
plt.title("Confusion Matrix – Validation")
plt.show()

import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# ===========================================================
# 1. Extract dataset
# ===========================================================
extract_path = os.path.join(os.getcwd(), "dataset_folder")
if not os.path.exists(extract_path):
    with zipfile.ZipFile("archive.zip", 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"✅ Dataset extracted to {extract_path}")
else:
    print(f"✅ Dataset already exists at {extract_path}")

# ===========================================================
# 2. Parameters
# ===========================================================
IMG_SIZE = (300, 300)  # ✅ Best for EfficientNet
BATCH_SIZE = 32
VAL_SPLIT = 0.2

# ===========================================================
# 3. Stronger Augmentation
# ===========================================================
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VAL_SPLIT,
    rotation_range=40,
    shear_range=0.3,
    zoom_range=(0.6, 1.4),
    width_shift_range=0.3,
    height_shift_range=0.3,
    channel_shift_range=30.0,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    os.path.join(extract_path, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    os.path.join(extract_path, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

print("✅ Class indices:", train_gen.class_indices)
print("📊 Train distribution:", dict(Counter(train_gen.classes)))
print("📊 Val distribution:", dict(Counter(val_gen.classes)))

# ===========================================================
# 4. Class Weights
# ===========================================================
class_weights = dict(enumerate(compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)))

# ===========================================================
# 5. Build EfficientNetV2S Model
# ===========================================================
def build_efficientnetv2_model(input_shape=(300, 300, 3), num_classes=3):
    base_model = EfficientNetV2S(weights="imagenet", include_top=False, input_tensor=Input(shape=input_shape))
    base_model.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation="softmax")(x)

    from tensorflow.keras.optimizers.schedules import CosineDecay
    lr_schedule = CosineDecay(initial_learning_rate=0.001, decay_steps=10000)
    model = Model(inputs=base_model.input, outputs=out)
    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss="categorical_crossentropy", metrics=["accuracy"])
    return model, base_model

model, base_model = build_efficientnetv2_model()
model.summary()

# ===========================================================
# 6. Callbacks
# ===========================================================
callbacks = [
    ModelCheckpoint("best_efficientnetv2s.h5", monitor="val_accuracy", save_best_only=True, mode="max", verbose=1),
    EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True, verbose=1)
]

# ===========================================================
# 7. Phase 1: Train Frozen Model
# ===========================================================
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,  # ✅ More epochs
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# ===========================================================
# 8. Phase 2: Fine-Tune more layers
# ===========================================================
for layer in base_model.layers[-60:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,  # ✅ More epochs
    initial_epoch=history1.epoch[-1] + 1,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# ===========================================================
# 9. Evaluation
# ===========================================================
val_loss, val_acc = model.evaluate(val_gen, verbose=0)
print(f"\n✅ Final validation accuracy: {val_acc:.2%}")

def plot_history(histories):
    plt.figure(figsize=(12, 4))
    for h in histories:
        plt.subplot(1, 2, 1)
        plt.plot(h.history['accuracy'])
        plt.plot(h.history['val_accuracy'])
    plt.title("Accuracy")
    plt.legend(['Train', 'Val'])
    plt.grid()

    for h in histories:
        plt.subplot(1, 2, 2)
        plt.plot(h.history['loss'])
        plt.plot(h.history['val_loss'])
    plt.title("Loss")
    plt.legend(['Train', 'Val'])
    plt.grid()
    plt.tight_layout()
    plt.show()

plot_history([history1, history2])

# ===========================================================
# 10. Confusion Matrix
# ===========================================================
val_gen.reset()
preds = model.predict(val_gen, verbose=0)
y_true = val_gen.classes
y_pred = np.argmax(preds, axis=1)

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=list(train_gen.class_indices.keys()))
disp.plot(cmap="Blues", colorbar=False)
plt.title("Confusion Matrix – Validation")
plt.show()

# ===========================================================
# 11. OPTIONAL TTA Function (+2% boost)
# ===========================================================
def predict_with_tta(model, generator, tta_steps=5):
    preds_tta = np.zeros((generator.samples, generator.num_classes))
    for i in range(tta_steps):
        generator.reset()
        preds_tta += model.predict(generator, verbose=0)
    preds_tta /= tta_steps
    return preds_tta

# Example usage:
# preds_tta = predict_with_tta(model, val_gen, tta_steps=5)
# y_pred_tta = np.argmax(preds_tta, axis=1)
# cm = confusion_matrix(y_true, y_pred_tta)
# disp = ConfusionMatrixDisplay(cm, display_labels=list(train_gen.class_indices.keys()))
# disp.plot(cmap="Blues", colorbar=False)
# plt.title("TTA Confusion Matrix – Validation")
# plt.show()

"""
Skin‑Tone Classification with EfficientNetV2‑S

"""


# 1. Imports

import os, zipfile, numpy as np, matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D,
                                     Dense, Dropout, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.optimizers.schedules import CosineDecay


# 2. Dataset (zip ⇒ folder if needed)

extract_path = os.path.join(os.getcwd(), "dataset_folder")
if not os.path.exists(extract_path):
    with zipfile.ZipFile("archive.zip") as zf:
        zf.extractall(extract_path)
    print(f"Dataset extracted to {extract_path}")
else:
    print(f"✅ Dataset already exists at {extract_path}")


# 3. Parameters & Augmentation

IMG_SIZE   = (300, 300)              # best for EfficientNetV2‑S
BATCH_SIZE = 32
VAL_SPLIT  = 0.20

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VAL_SPLIT,
    rotation_range=40,
    shear_range=0.3,
    zoom_range=(0.6, 1.4),
    width_shift_range=0.3,
    height_shift_range=0.3,
    channel_shift_range=30.0,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
)

train_gen = datagen.flow_from_directory(
    os.path.join(extract_path, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
)

val_gen = datagen.flow_from_directory(
    os.path.join(extract_path, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False,
)

print("Class indices:", train_gen.class_indices)
print("Train distribution:", dict(Counter(train_gen.classes)))
print("Val distribution:", dict(Counter(val_gen.classes)))


# 4. Class Weights (handle any imbalance)

class_weights = dict(
    enumerate(
        compute_class_weight(
            class_weight="balanced",
            classes=np.unique(train_gen.classes),
            y=train_gen.classes,
        )
    )
)


# 5. Build EfficientNetV2‑S model

def build_model(input_shape=(300, 300, 3), n_classes=3):
    base = EfficientNetV2S(
        weights="imagenet", include_top=False, input_tensor=Input(shape=input_shape)
    )
    base.trainable = False                              # Phase‑1 frozen

    x = GlobalAveragePooling2D()(base.output)
    x = BatchNormalization()(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.5)(x)
    out = Dense(n_classes, activation="softmax")(x)

    lr_sched = CosineDecay(initial_learning_rate=1e-3, decay_steps=10_000)
    model = Model(base.input, out)
    model.compile(optimizer=Adam(lr_sched),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model, base

model, base_model = build_model()
model.summary()


# 6. Callbacks

callbacks = [
    ModelCheckpoint("best_efficientnetv2s.h5",
                    monitor="val_accuracy",
                    save_best_only=True,
                    mode="max",
                    verbose=1),
    EarlyStopping(monitor="val_loss",
                  patience=6,
                  restore_best_weights=True,
                  verbose=1),
]


# 7. Phase‑1 Training (feature extractor)

history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1,
)


# 8. Phase‑2 Fine‑Tuning (unfreeze last 60 layers)

for layer in base_model.layers[-60:]:
    layer.trainable = True

model.compile(optimizer=Adam(1e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    initial_epoch=len(history1.epoch),
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1,
)


# 9. Metrics Plot

def plot_history(histories):
    plt.figure(figsize=(12, 4))
    # --- Accuracy
    plt.subplot(1, 2, 1)
    for h in histories:
        plt.plot(h.history["accuracy"])
        plt.plot(h.history["val_accuracy"])
    plt.title("Accuracy")
    plt.grid(True)
    # --- Loss
    plt.subplot(1, 2, 2)
    for h in histories:
        plt.plot(h.history["loss"])
        plt.plot(h.history["val_loss"])
    plt.title("Loss")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_history([history1, history2])


# 10. Evaluation + Confusion Matrix (label‑remap)

val_loss, val_acc = model.evaluate(val_gen, verbose=0)
print(f"Final validation accuracy: {val_acc:.2%}")

# ── Predictions
val_gen.reset()
preds   = model.predict(val_gen, verbose=0)
y_true  = val_gen.classes
y_pred  = np.argmax(preds, axis=1)

# ── Human‑readable label mapping
orig_to_readable = {'Black': 'dark', 'Brown': 'medium', 'White': 'light'}
readable_labels  = [orig_to_readable[k] for k in train_gen.class_indices.keys()]

cm   = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=readable_labels)
disp.plot(cmap="Blues", colorbar=False)
plt.title("Confusion Matrix – Validation")
plt.show()


# 11. (Optional) Test‑Time Augmentation helper

def predict_with_tta(model, generator, tta_steps=5):
    """Return averaged probabilities across `tta_steps` passes."""
    acc_probs = np.zeros((generator.samples, generator.num_classes))
    for _ in range(tta_steps):
        generator.reset()
        acc_probs += model.predict(generator, verbose=0)
    return acc_probs / tta_steps

!pip install tensorflow==2.12.0 --upgrade

"""
EfficientNetV2‑S skin‑tone classifier (improved)
Folders: dataset_folder/train/Black, Brown, White
Human‑readable labels: dark / medium / light
"""

# ===========================================================
# 1. Imports & dataset unzip (same as before)
# ===========================================================
import os, zipfile, numpy as np, matplotlib.pyplot as plt, tensorflow as tf
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense,
                                     Dropout, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping,
                                        ReduceLROnPlateau)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

extract_path = os.path.join(os.getcwd(), "dataset_folder")
if not os.path.exists(extract_path):
    with zipfile.ZipFile("archive.zip") as zf:
        zf.extractall(extract_path)

# ===========================================================
# 2. Parameters
# ===========================================================
IMG_SIZE   = (300, 300)
BATCH_SIZE = 32
VAL_SPLIT  = 0.2

# ===========================================================
# 3. Moderated augmentation
# ===========================================================
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VAL_SPLIT,
    rotation_range=30,
    shear_range=0.25,
    zoom_range=(0.75, 1.25),
    width_shift_range=0.2,
    height_shift_range=0.2,
    channel_shift_range=20.0,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
)

train_gen = datagen.flow_from_directory(
    os.path.join(extract_path, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
)

val_gen = datagen.flow_from_directory(
    os.path.join(extract_path, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False,
)

print("✅ Class indices:", train_gen.class_indices)
print("📊 Train distribution:", dict(Counter(train_gen.classes)))
print("📊 Val   distribution:", dict(Counter(val_gen.classes)))

# ===========================================================
# 4. Class weights
# ===========================================================
class_weights = dict(
    enumerate(
        compute_class_weight("balanced",
                             classes=np.unique(train_gen.classes),
                             y=train_gen.classes)
    )
)

# ===========================================================
# 5. Focal‑loss with label smoothing
# ===========================================================
def focal_loss(alpha=1.0, gamma=2.0, label_smoothing=0.05):
    alpha = tf.constant(alpha, dtype=tf.float32)
    def loss(y_true, y_pred):
        y_true   = y_true * (1 - label_smoothing) + label_smoothing / 3
        y_pred   = tf.clip_by_value(y_pred, 1e-7, 1-1e-7)
        cross_ce = -y_true * tf.math.log(y_pred)
        weight   = alpha * tf.pow(1 - y_pred, gamma)
        return tf.reduce_sum(weight * cross_ce, axis=-1)
    return loss

alpha_vec = [1.0, 3.0, 1.0]      # amplify medium class (index 1)

# ===========================================================
# 6. Build EfficientNetV2‑S model
# ===========================================================
def build_model():
    base = EfficientNetV2S(weights="imagenet",
                           include_top=False,
                           input_tensor=Input(shape=(*IMG_SIZE, 3)))
    base.trainable = False
    x = GlobalAveragePooling2D()(base.output)
    x = BatchNormalization()(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.5)(x)
    out = Dense(3, activation="softmax")(x)

    lr_sched = CosineDecayRestarts(initial_learning_rate=1e-3,
                                   first_decay_steps=5*len(train_gen),
                                   t_mul=2.0, m_mul=0.5)

    model = Model(base.input, out)
    model.compile(optimizer=Adam(lr_sched),
                  loss=focal_loss(alpha_vec, gamma=2.0, label_smoothing=0.05),
                  metrics=["accuracy"])
    return model, base

model, base_model = build_model()
model.summary()

# ===========================================================
# 7. Callbacks
# ===========================================================
cbs = [
    ModelCheckpoint("best_effnetv2s.h5", monitor="val_accuracy",
                    save_best_only=True, mode="max", verbose=1),
    EarlyStopping(monitor="val_loss", patience=8,
                  restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2,
                      patience=4, min_lr=1e-6, verbose=1),
]

# ===========================================================
# 8. Phase‑1: frozen feature extractor
# ===========================================================
history1 = model.fit(train_gen,
                     validation_data=val_gen,
                     epochs=20,
                     class_weight=class_weights,
                     callbacks=cbs,
                     verbose=1)

# ===========================================================
# 9. Phase‑2: fine‑tune last 120 layers
# ===========================================================
for layer in base_model.layers[-120:]:
    layer.trainable = True
model.compile(optimizer=Adam(3e-6),
              loss=focal_loss(alpha_vec, 2.0, 0.05),
              metrics=["accuracy"])

history2 = model.fit(train_gen,
                     validation_data=val_gen,
                     epochs=25,
                     initial_epoch=len(history1.epoch),
                     class_weight=class_weights,
                     callbacks=cbs,
                     verbose=1)

# ===========================================================
# 10. Evaluation & plots
# ===========================================================
val_loss, val_acc = model.evaluate(val_gen, verbose=0)
print(f"\n🔎 Final validation accuracy: {val_acc:.2%}")

# training curves
def plot_hist(*hists):
    plt.figure(figsize=(12, 4))
    # accuracy
    plt.subplot(1, 2, 1)
    for h in hists:
        plt.plot(h.history["accuracy"])
        plt.plot(h.history["val_accuracy"])
    plt.title("Accuracy"); plt.grid(True)
    # loss
    plt.subplot(1, 2, 2)
    for h in hists:
        plt.plot(h.history["loss"])
        plt.plot(h.history["val_loss"])
    plt.title("Loss"); plt.grid(True)
    plt.tight_layout(); plt.show()

plot_hist(history1, history2)

# ===========================================================
# 11. Confusion matrix (renamed)
# ===========================================================
val_gen.reset()
preds   = model.predict(val_gen, verbose=0)
y_true  = val_gen.classes
y_pred  = np.argmax(preds, axis=1)

label_map = {"Black": "dark", "Brown": "medium", "White": "light"}
readable  = [label_map[k] for k in train_gen.class_indices.keys()]

cm   = confusion_matrix(y_true, y_pred)
ConfusionMatrixDisplay(cm, display_labels=readable).plot(
    cmap="Blues", colorbar=False)
plt.title("Confusion Matrix – Validation")
plt.show()